In [1]:
import pandas as pd
import numpy as np
from statistics import mean
import math

pd.set_option('display.max_columns', 30)

types = {'id': int, 'provincia': str}

training_data = pd.read_csv(r'../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','tipodepropiedad' ,'direccion'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv', dtype = types, usecols=['id','provincia','tipodepropiedad', 'direccion'])

training_data.head()

/home/mati/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/mati/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,id,tipodepropiedad,direccion,provincia
0,254099,Apartamento,Avenida Division del Norte 2005,Distrito Federal
1,53461,Casa en condominio,AV. MEXICO,Distrito Federal
2,247984,Casa,Urbi Tonala,Jalisco
3,209067,Casa,IGNACIO MANUEL ALTAMIRANO 128,Edo. de México
4,185997,Apartamento,PASEOS DEL SOL,Jalisco


In [2]:
training_data["direccion"] = training_data["direccion"].apply(lambda x: str(x).lower())
evaluation_data["direccion"] = evaluation_data["direccion"].apply(lambda x: str(x).lower())
training_data["tipodepropiedad"] = training_data["tipodepropiedad"].apply(lambda x: str(x).lower())
evaluation_data["tipodepropiedad"] = evaluation_data["tipodepropiedad"].apply(lambda x: str(x).lower())

In [7]:
training_data['sum'] = 1
unique = training_data.groupby(['provincia','direccion','tipodepropiedad']).agg(unique_dirs = ('sum','sum')).reset_index()
unique['sum'] = 1
unique = unique.groupby(['provincia']).agg(unique_dirs = ('sum','sum'), total_dirs =('unique_dirs','sum') ).reset_index()
unique['porcentaje_dir_unicas_provincia'] = unique['unique_dirs']/unique['total_dirs']
unique['log_dir_unicas_provincia'] = unique['unique_dirs'].apply(math.log)
unique = unique.drop(['unique_dirs','total_dirs'],axis=1)
td = training_data.merge(unique, on='provincia', how = 'outer')
td.head()

,id,tipodepropiedad,direccion,provincia,sum,porcentaje_dir_unicas_provincia,log_dir_unicas_provincia
0,254099,apartamento,avenida division del norte 2005,Distrito Federal,1,0.6529,10.555396
1,53461,casa en condominio,av. mexico,Distrito Federal,1,0.6529,10.555396
2,126147,apartamento,condominio tlalpan 2b,Distrito Federal,1,0.6529,10.555396
3,130216,casa,"matias romero, colonia del valle",Distrito Federal,1,0.6529,10.555396
4,73348,apartamento,pazaje santa cruz,Distrito Federal,1,0.6529,10.555396


In [8]:
evaluation_data['sum'] = 1
unique = evaluation_data.groupby(['provincia','direccion','tipodepropiedad']).agg(unique_dirs = ('sum','sum')).reset_index()
unique['sum'] = 1
unique = unique.groupby(['provincia']).agg(unique_dirs = ('sum','sum'), total_dirs =('unique_dirs','sum') ).reset_index()
unique['porcentaje_dir_unicas_provincia'] = unique['unique_dirs']/unique['total_dirs']
unique['log_dir_unicas_provincia'] = unique['unique_dirs'].apply(math.log)
unique = unique.drop(['unique_dirs','total_dirs'],axis=1)
ed = evaluation_data.merge(unique, on='provincia',how ='outer')
ed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 7 columns):
id                                 60000 non-null int64
tipodepropiedad                    60000 non-null object
direccion                          60000 non-null object
provincia                          59958 non-null object
sum                                60000 non-null int64
porcentaje_dir_unicas_provincia    59958 non-null float64
log_dir_unicas_provincia           59958 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 3.7+ MB


In [9]:
td.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 7 columns):
id                                 240000 non-null int64
tipodepropiedad                    240000 non-null object
direccion                          240000 non-null object
provincia                          239845 non-null object
sum                                240000 non-null int64
porcentaje_dir_unicas_provincia    239845 non-null float64
log_dir_unicas_provincia           239845 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 14.6+ MB


In [6]:
evaluation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 5 columns):
id                 60000 non-null int64
tipodepropiedad    60000 non-null object
direccion          60000 non-null object
provincia          59958 non-null object
sum                60000 non-null int64
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [28]:
td = td.drop(['tipodepropiedad','direccion','provincia','sum'],axis = 1)
ed = ed.drop(['tipodepropiedad','direccion','provincia','sum'], axis = 1)
td.head()

,id,porcentaje_dir_unicas_provincia,log_dir_unicas_provincia
0,254099,0.808692,10.653771
1,53461,0.808692,10.653771
2,126147,0.808692,10.653771
3,130216,0.808692,10.653771
4,73348,0.808692,10.653771


In [26]:
td.to_csv('../../res/ftr/direcciones_unicas_train.csv')
ed.to_csv('../../res/ftr/direcciones_unicas_evaluation.csv')